In [1]:
import datetime as dt
from collections import OrderedDict
from pathlib import Path
import sys

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import curio
import curio_http

ROOT = Path('../')
DATA_DIR = ROOT/'data'
OUT_DIR = ROOT/'output'

sys.path.append(str(ROOT))

%load_ext autoreload
%autoreload 2

import countdowner as cd

In [2]:
# Read a watchlist

w = cd.read_watchlist(DATA_DIR/'test_watchlist.yaml')
w


{'email_address': 'brainbummer@mailinator.com',
 'name': 'test',
 'products':                          description stock_code
 0                     organic cheese     281739
 1                       GB chocolate     260803
 2                     Lupi olive oil     701829
 3      Earthcare double toilet paper     381895
 4                      Dijon mustard     700630
 5                whole grain mustard     700631
 6            Pics peanut butter 380g     360257
 7           Pics peanut butter 1000g     887052
 8                      coconut cream     271818
 9     Ecostore dishwash liquid 500mL     701108
 10   Ecostore dishwash liquid 1000mL     797559
 11   Earthwise dishwash liquid 500mL     701227
 12  Earthwise dishwash liquid 1000mL     504018}

In [3]:
# Get and parse product info

code = w['products']['stock_code'].iat[0]
html = cd.get_product(code)
cd.parse_product(html)

OrderedDict([('stock_code', '281739'),
             ('name', 'Mainland Cheese Block Organic Cheddar 500g'),
             ('description',
              'Mainland organic cheddar is a mild cheddar cheese made from certified organic milk.'),
             ('size', '500g'),
             ('on_sale', False),
             ('sale_price', None),
             ('price', 11.0),
             ('discount', None),
             ('unit_price', '$22.00/1KG'),
             ('datetime', '2017-05-30T12:08:47')])

In [4]:
# Collect info for several products

codes = w['products']['stock_code'].values
%time f = cd.collect_products(codes)
f

CPU times: user 1.03 s, sys: 4 ms, total: 1.04 s
Wall time: 5.12 s


,stock_code,name,description,size,on_sale,sale_price,price,discount,unit_price,datetime
0,281739,Mainland Cheese Block Organic Cheddar 500g,Mainland organic cheddar is a mild cheddar che...,500g,False,NaN,11.00,NaN,$22.00/1KG,2017-05-30 12:08:47
1,260803,Green & Blacks Chocolate Block Organic Dark Ch...,None,100g,True,3.50,3.89,0.100257,$3.50/100G,2017-05-30 12:08:47
2,701829,Lupi Olive Oil Organic Extra Virgin 750ml,None,750ml,False,NaN,15.00,NaN,$2.00/100ML,2017-05-30 12:08:48
3,381895,Earthcare Toilet Paper 6pk Double Lenght Sky ...,None,6pk,False,NaN,5.00,NaN,$0.19/100SS,2017-05-30 12:08:48
4,700630,Macro Organic Mustard Dijon 200g,None,200g,True,2.99,3.49,0.143266,None,2017-05-30 12:08:49
5,700631,Macro Organic Mustard Wholegrain 200g,None,200g,True,2.99,3.49,0.143266,None,2017-05-30 12:08:49
6,360257,Pics Peanut Butter Crunchy 380g,None,380g,False,NaN,6.50,NaN,$1.71/100G,2017-05-30 12:08:49
7,887052,Pics Peanut Butter Crunchy 1kg,None,1kg,True,16.50,18.49,0.107626,$16.50/1KG,2017-05-30 12:08:50
8,271818,Ceres Organics Coconut Cream Creamy & Unsweete...,Creamy and unsweetened,can 400ml,False,NaN,4.25,NaN,$1.06/100G,2017-05-30 12:08:50
9,701108,Ecostore Dishwash Liquid Grapefruit 500ml,Powerful formulation for sparkling clean dishe...,500ml,True,3.19,3.95,0.192405,$0.64/100ML,2017-05-30 12:08:51


In [5]:
# Collect info for several products asynchronously

codes = w['products']['stock_code'].values
%time f = cd.collect_products_a(codes)
f

CPU times: user 972 ms, sys: 16 ms, total: 988 ms
Wall time: 1.28 s


,stock_code,name,description,size,on_sale,sale_price,price,discount,unit_price,datetime
8,271818,Ceres Organics Coconut Cream Creamy & Unsweete...,Creamy and unsweetened,can 400ml,False,NaN,4.25,NaN,$1.06/100G,2017-05-30 12:08:53
3,381895,Earthcare Toilet Paper 6pk Double Lenght Sky ...,None,6pk,False,NaN,5.00,NaN,$0.19/100SS,2017-05-30 12:08:53
11,701227,Earthwise Dishwash Liquid 500ml,None,500ml,False,NaN,3.99,NaN,$0.80/100ML,2017-05-30 12:08:53
12,504018,Earthwise Dishwash Liquid Pink Grapefruit 1l,None,1l,False,NaN,7.00,NaN,$0.70/100ML,2017-05-30 12:08:53
10,797559,Ecostore Dishwash Liquid Grapefruit 1l,Powerful formulation for sparkling clean dishe...,1l,False,NaN,7.39,NaN,$0.74/100ML,2017-05-30 12:08:53
9,701108,Ecostore Dishwash Liquid Grapefruit 500ml,Powerful formulation for sparkling clean dishe...,500ml,True,3.19,3.95,0.192405,$0.64/100ML,2017-05-30 12:08:53
1,260803,Green & Blacks Chocolate Block Organic Dark Ch...,None,100g,True,3.50,3.89,0.100257,$3.50/100G,2017-05-30 12:08:52
2,701829,Lupi Olive Oil Organic Extra Virgin 750ml,None,750ml,False,NaN,15.00,NaN,$2.00/100ML,2017-05-30 12:08:52
4,700630,Macro Organic Mustard Dijon 200g,None,200g,True,2.99,3.49,0.143266,None,2017-05-30 12:08:53
5,700631,Macro Organic Mustard Wholegrain 200g,None,200g,True,2.99,3.49,0.143266,None,2017-05-30 12:08:53


In [6]:
# Filter sales

g = cd.filter_sales(f)
g

,name,sale_price,price,discount
9,Ecostore Dishwash Liquid Grapefruit 500ml,3.19,3.95,0.192405
1,Green & Blacks Chocolate Block Organic Dark Ch...,3.50,3.89,0.100257
4,Macro Organic Mustard Dijon 200g,2.99,3.49,0.143266
5,Macro Organic Mustard Wholegrain 200g,2.99,3.49,0.143266
7,Pics Peanut Butter Crunchy 1kg,16.50,18.49,0.107626


In [7]:
# Email sales

domain = cd.get_secret('mailgun_domain')
key = cd.get_secret('mailgun_api_key')
cd.email(g, 'brainbummer@mailinator.com', domain, key)

<Response [200]>

In [8]:
# Run pipeline: read watchlist, get product info, write product info, 
# filter sales, and email results

%time cd.run_pipeline(DATA_DIR/'test_watchlist.yaml', OUT_DIR, domain, key)

CPU times: user 1.01 s, sys: 0 ns, total: 1.01 s
Wall time: 2.04 s
